In [1]:
# Declarations
import glob
import json
import os
import numpy as np
import pandas as pd

from os.path import basename

In [2]:
# DataFrame name json file
nameFile = []
pathFile = []
rootFile = []

path = os.getcwd()
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".json"):
            nameFile.append(file)
            pathFile.append(os.path.join(root, file))
            rootFile.append(root)
            
nameFileS = pd.Series(nameFile)
pathFileS = pd.Series(pathFile)
rootFileS = pd.Series(rootFile)

dfNF = pd.concat([nameFileS, pathFileS, rootFileS], axis=1)
dfNF.columns = ['FILE_NAME', 'FILE_PATH', 'FILE_ROOT']

In [3]:
# .json file loop
df = pd.DataFrame(columns=['DEVICE','BOARD','FUNCTION','DATA_TYPE',
                           'NUM_S','RANGE_S','NUM_SI','RANGE_SI',
                           'NUM_ARR','RANGE_ARR'])

listD = ["Bambu"] # Inserisci i dispositivi che vuoi analizzare
listDT = ["int8_t"] # Inserisci i data type che vuoi analizzare

for i in range(0, len(pathFile)):
    pF = pathFile[i]

    with open(pF, 'r') as jf:
        data = jf.read()
    obj = json.loads(data)

    if os.name == 'nt':
        spR = rootFile[i].split('\\')
    else:
        spR = rootFile[i].split('/')
    function = spR[-1]
    board = spR[-3]

    #Device loop
    deviceList = list(obj.keys())
    deviceListR = []
    for u in range(0, len(deviceList)):
        if deviceList[u] in listD:
            deviceListR.append(deviceList[u])
    
    for n in range(0, len(deviceListR)):
        # Data_type loop
        device = deviceListR[n]
        dtarr = list(obj[deviceListR[n]].keys())
        
        dtarrR = []
        for u in range(0, len(dtarr)):
            if dtarr[u] in listDT:
                dtarrR.append(dtarr[u])
        
        for j in range(0, len(dtarrR)):
            dataype = dtarrR[j]
            lsubNP = (list(obj[deviceListR[n]][dtarrR[j]].keys()))
            lsubVP = (list(obj[deviceListR[n]][dtarrR[j]].values()))

            # sublist dimension
            ltot = len(lsubNP)

            # how many array are present
            na = 0
            fl = 0
            
            ark = 0
            
            for k in range(0, ltot): 
                #print(function, fl)
                if "[" in lsubNP[k]:
                    na = na + 1
                    if fl == 0:
                        ark = k
                        fl = 1

            # how many scalar index are present
            nsi = 0
            for k in range(0, ltot):
                flag = 0
                if not "[" in lsubNP[k]:
                    for keysl in lsubNP:
                        if (lsubNP[k] in keysl) and (not lsubNP[k] is keysl) and (flag == 0):          
                            flag = 1
                            nsi = nsi + 1           

            # how many scalar are present
            ns = ltot - na - nsi

            # check if is it negative
            newstr = lsubVP[ark].replace("[", "")
            newstr = newstr.replace("]", "")
            newstr = newstr.split(",")[0]
            num = float(newstr)
            ranArr = 0
            if num < 0:
                ranArr = 1
            else:
                ranArr = 0

            df = df.append({'DEVICE': device, 'BOARD': board, 'FUNCTION': function, 'DATA_TYPE': dataype,
                            'NUM_S': ns, 'RANGE_S': 0,'NUM_SI': nsi,'RANGE_SI': 0,
                            'NUM_ARR': na,'RANGE_ARR': ranArr}, 
                            ignore_index=True)

In [ ]:
dfS = pd.read_csv("dfS.csv", sep=';')
dfSI = pd.read_csv("dfSI.csv", sep=';')

In [4]:
dfS = pd.read_csv("dfS_tot.csv", sep=';')
dfSI = pd.read_csv("dfSI_tot.csv", sep=';')

In [5]:
del dfS['Unnamed: 0']
del dfSI['Unnamed: 0']

In [6]:
columnsS = ['DEVICE','BOARD','FUNCTION','DATA_TYPE','RANGE_S']
columnsSI = ['DEVICE','BOARD','FUNCTION','DATA_TYPE','RANGE_SI']
dfS = dfS[columnsS]
dfSI = dfSI[columnsSI]

In [7]:
lent = len(dfS['RANGE_S'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfS['DEVICE'][i]) & 
           (df['BOARD'] == dfS['BOARD'][i]) &
           (df['FUNCTION'] == dfS['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfS['DATA_TYPE'][i]), 'RANGE_S'] = dfS['RANGE_S'][i]

In [8]:
lent = len(dfSI['RANGE_SI'].tolist())
for i in range(0, lent):
    df.loc[(df['DEVICE'] == dfSI['DEVICE'][i]) & 
           (df['BOARD'] == dfSI['BOARD'][i]) & 
           (df['FUNCTION'] == dfSI['FUNCTION'][i]) & 
           (df['DATA_TYPE'] == dfSI['DATA_TYPE'][i]), 'RANGE_SI'] = dfSI['RANGE_SI'][i]

In [9]:
df.to_csv("RangeValueClassification_tot.csv", sep=';')